# Orientation - Working Software, *Now*

There is widespread consensus that API Servers are strategic to the business, required for mobile apps and internal / external systems integration.

The problem is that they are time-consuming and costly to develop (routes, controllers, ORMs, tedious domain-specific logic...). This reduces strategic business agility.  At the tactical level, UI development can be blocked on API coding.

> But imagine that you could create an API Server instantly, from a database?  And that logic could be specified with spreadsheet-like rules?  Plus an instant Web App to engage Business Users?

That's the vision driving API Logic Server - Working Software, *Now*.  In this 10-20 minute tutorial, we'll **create**, **explore** and **customize** the 3 main elements of an API Logic Server:
1. A JSON:**API** for a database
1. Business **Logic**
1. A Basic Web App

## Configure Workspace

You are in a JupyterLab Notebook (see Links, at end).  It's provides a "notebook" (a combination of markdown text and code), terminal windows, file explorers and text editors.  It's installed in a docker container deployed to the cloud, courtesy of MyBinder.  What will they think of next!

Installation has already been completed for both Python and ApiLogicServer.
ApiLogicServer installs with a [Northwind sample database](https://github.com/valhuber/ApiLogicServer/wiki/Sample-Database),
which we'll be using in this Tutorial.

> A **Build** dialog may occur - just cancel

Arrange your workspace:

1. Open the Notebook
1. **File > New** to open 2 **Terminal** windows
   * **Drag** them to arrange your workspace as shown below
2. Click the icon on the left panel to view the **Table of Contents**
   * This is a great way to maintain context
4. When you're ready, **advance:**
   * Scroll down, or click the down arrow, or select a section at left.

<figure><img src="https://github.com/valhuber/ApiLogicServer/blob/main/images/tutorial/notebook-workspace.png?raw=true"></figure>

If you are considering Jupyter notebooks for your own no-install demos,  you can explore how this demo was built on GitHub (link at end).

# Create ApiLogicServer
In this step, we'll use the **ApiLogicServer CLI** to create and run an **api_logic_server project** (an **API** and **Web App**, backed by underlying **Logic**):

To create the ApiLogicServer:

1. Paste the following into the left Terminal window to run the ApiLogicServer CLI:

```bash
ApiLogicServer run
```

2. Accept the default `Database URL` parameter to use the supplied sample database.

   * After this Tutorial, you can try it with your own database, by specifying a SQLAlchemy URL (see links below).

### Quite A Lot Just Happened
Quite a lot just happened that substantially accelerates your project.  The process is shown in the diagram below, mapping *What You Do (in green:* A, B, C -- you just did step B), what the *system does* (the labeled arrows -- it just did steps 1 and 2) onto a classic *n-tiered modern software architecture (in blue):*

   
|What *You* Do| What *System* Does   | *Why* It Matters  |
| :-------------- |:--------------| :------| 
| **A) Create Data Base**<br>(Presuppied for demo) | ** | Use existing tools and procedures |
| **B)&nbsp;ApiLogicServer&nbsp;run**<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| **1 Instant API:** From schema, creates an endpoint for each table<br>CRUD **Data Access** automation, including filtering, pagination and multi-table access | **Ready to run:** Custom UI development is ready to go <br> not blocked on API coding (routes, controllers, ORMs...)|
| |  **2 Instant Web App:** Multi-page, multi-table | **Engage business users** with working software, **early** in the project|
| **C) Declare Logic**| Executes **spreadsheet-like rules** that automate the cocktail napkin spec | **40X more concise**, *customizable* with Python  |
       
<figure><img src="https://github.com/valhuber/ApiLogicServer/blob/main/images/tutorial/dev-process.png?raw=true"></figure>


# 1. Explore the API

In many projects, User Interface development is blocked by waiting on API creation.

The ```run``` command not only _created_ the project, but it _ran_ ```python api_logic_server/api_logic_server_run.py``` to start the server.  You have a working API.  Let's explore it.

> **Key Takeaway:** ApiLogicServer unblocks UI development with instant API creation.

## Open API (Swagger) - LOCAL ONLY

You can explore the created API with Open API (Swagger).
If you were running locally, you could explore the api at `https://localhost:5000`.

> **Not working for cloud version:** you need to be running locally to see this screen:

<figure><img src="https://github.com/valhuber/ApiLogicServer/raw/main/images/tutorial/explore-api.png"></figure>


## Get

In addition to Open API, we can use curl.

1. Paste this into the right terminal window:

```bash
curl -X GET "http://localhost:5000/Order\
?page%5Boffset%5D=0\
&page%5Blimit%5D=2\
&sort=Id%2CCustomerId%2CEmployeeId\
&filter%5BCustomerId%5D=ALFKI"\
 -H  "accept: application/vnd.api+json" \
 -H  "Content-Type: application/vnd.api+json"
```
2. Observe the response in the Terminal window
   * ignore the logging in the server Terminal window

### Pagination, Filtering
In the `GET` request, observe:
* `page offset` - pagination support
* `filter` - autmatic support for filtering rows


### Configurable Fields, Joins
JSON:APIs are **client configurable.**  This is valuable because:

* Client Configurable APIs can **reduce network traffic** - a _single call_, instead of _multiple calls_ on predefined APIs that don't return quite the correct data
* **Reduced organizational dependency** on server development - instead of a waiting on creation of a wide number of endpoints, clients can configure their own APIs

In the example below, observe the following in the `GET` request:
* `fields` to designate the fields to be returned
* `include` to designate **related data** 

1. Paste this into the right Terminal window:

```bash
curl -X GET "http://localhost:5000/Order\
?include=OrderDetailList\
&fields%5BOrder%5D=Id%2CCustomerId%2CEmployeeId%2COrderDate%2CAmountTotal\
&page%5Boffset%5D=0\
&page%5Blimit%5D=2\
&sort=Id%2CCustomerId%2CEmployeeId\
&filter%5BCustomerId%5D=ALFKI"\
 -H  "accept: application/vnd.api+json" \
 -H  "Content-Type: application/vnd.api+json"
```

2. In the response, observe the `"included": [` tag - the list of related `OrderDetail` records, and the specified fields


## Patch: Logic Enabled Updates
Your API also includes services for insert, update and delete. We can use curl to test the update.

1. Paste the following into the right Terminal window, and observe the expected failure response:

```bash
curl -vX PATCH "http://localhost:5000/Customer/ALFKI/" -H  "accept: application/vnd.api+json" -H  "Content-Type: application/json"  -d '
{
  "data": {
     "attributes": {
        "CreditLimit": "100"
     },
  "type": "Customer",
  "id": "ALFKI"
}}'
```

2. Observe the update request reports the violation from our predefined rules:

```bash
balance (1016.0000000000) exceeds credit (100)
```

This was entirely intentional, to illustrate the transaction logic underlying the API.  Let's explore that.

# 2. Explore Transaction Logic

The classic definition of Business Logic covers 2 aspects:
1. **Data Access -** this was automated in the `ApiLogicServer run` step, which created not only the API, but also the data access (based on SQLAlchemy)
2. **Transaction logic -** multi-table derivations, constraints,
and actions such as sending mail or messages - is a significant
aspect of any database oriented system, nearly half

> Logic is the iceberg under the surface of the API.
> 
> ApiLogicServer provides a unique declarative, rules-based approach to Transaction Logic.

## Cocktail Napkin Explosion

It is striking how a small "cocktail napkin specification" can balloon into
hundreds of lines of code:

<figure><img src="https://github.com/valhuber/LogicBank/raw/main/images/overview/rules-vs-code.png"></figure>

> Implementing logic by conventional *procedural code is slow, error prone, and painful to maintain.*

## Logic is _Declarative_

Api Logic Server dramatically improves conciseness, quality and maintainability, by introducing a signicant innovation for addressing transaction logic: **Extensible Declarative Logic** based on LogicBank.  This approach is based on two things:

* **Rules - 40X** more concise using a declarative, spreadsheet-like paradigm, and

* **Python - control and extensibility,** using standard tools and techniques

  > **Note:** in this tutorial, the API and Web App were created strictly from the data model.
The logic, however, is injected so you can explore it.

Rules are _not_ simply procedural event handlers.  They are spreadsheet-like expressions for
constraints and derivations applied on update.  The process is follows:

1. **Design** as you normally would, specify the behavior - the "cocktail napkin spec" below
2. **Declare** rules to express your design
   * You will find that there is a very close mapping from your design to the rule declarations, as shown below
   * These 5 rules replace all the Python code shown above - 200 lines
   * Rule order is irrelevant - rule execution is automatically ordered by dependencies.  To illustrate, the business-oriented rules are reversed to a developers-oriented perspective that reflects execution order.  
3. **Extend with Python** as we'll show later

<figure><img src="https://github.com/valhuber/ApiLogicServer/raw/main/images/tutorial/declare-5-rules.png"></figure>

This table summarizes the **key declarative / procedural differences:**

| Characteristic | Procedural   | Declarative  | Why It Matters |
| :-------------: |:-------------:| :-----:| :-----:|
| **Reuse**     | Not Automatic | Automatic - all Use Cases | 40X Code Reduction |
| **Invocation** | Passive - only if called | Active - call not required | Quality |
| **Ordering** | Manual | Automatic | Agile Maintenance |
| **Optimizations**      | Manual     |   Automatic | Agile Design |

Unlike code, you do not call the rules directly.
The rules engine in Logic Bank listens for SQLAlchemy ```before_flush``` events,
and applies the applicable rules in an order that reflects their dependencies.

> For more information on rules, see the links at the end.

The constraint rule (line 84, above) is what caused the update error response above -
the ```Balance``` exceeded the altered ```CreditLimit```.  The transaction is rolled back.


### Perspective: 40X
Let's pause for some perspective: *what is **40X** more concise?*

A jetliner flies *4X* the speed of Charles Lindbergh's Spirit of St. Louis.  To get to 40X, you need a scram jet - the same as the degree of conciseness of legacy procedural logic vs. spreadsheet-like rules.

So, for nearly half your system, you are writing a rule instead of nearly a page of code. See the constrast in the links, at end.

<figure><img src="https://github.com/valhuber/ApiLogicServer/raw/main/images/tutorial/technology-matters.png"></figure>

## Explore Types, Examples

> **Key Takeaway:** Logic and Web Apps are familiar.  Logic, however, is perhaps  unexpected.  It's a new innovation, that delivers enormous value - so you will find it valuable to spend a few hours learning about it.

>> To learn more about Logic, see the links at the end, including a complete list of rules, and examples.

# 3. Explore the Basic Web App

UI development takes time.  That's a problem since:
* Such effort may not be warranted for admin "back office" screens,
and
  
* [Agile approaches](https://agilemanifesto.org) depend on getting _working
software_ soon, to drive _collaboration and iteration_.

ApiLogicServer CLI therefore creates working software _now_ - not just an API, but also
multi-page, multi-table applications as shown below:

1. **Multi-page:** apps include 1 page per table

2. **Multi-table:** pages include ``related_views`` for each related child table, and join in parent data

3. **Favorite fields first:** first-displayed field is "name", or `contains` "name" (configurable)

4. **Predictive joins:** favorite field of each parent is shown (product *name* - not product *id*)

5. **Ids last:** such boring fields are not shown on lists, and at the end on other pages

<figure><img src="https://raw.githubusercontent.com/valhuber/fab-quick-start/master/images/generated-page.png"></figure>

**If running locally** (this does not work in the MyBinder Cloud version), start the application:
```
cd ApiLogicServer/api_logic_server
python ui/basic_web_app/run.py
```

> Before running, you must Create Admin Data for Flask App Builder (see links).

# Customize With Python

Recall using API Logic Server CLI:
```
ApiLogicServer run
```
created a complete, executable ApiLogicServer in a directory called ```api_logic_server```.  On your own machine, you could open the project in an IDE (PyCharm, VSCode, etc) or an Editor (Atom, Text Wrangler, etc).  It looks like this in PyCharm:

<figure><img src="https://github.com/valhuber/ApiLogicServer/blob/main/images/generated-project.png?raw=true"></figure>

ApiLogicServer CLI introspected your database and created a `database/models.py` file (for SQLAlchemy), and files that declare your `api` and `ui/basic-web-app`.  You can edit those files to customize your server, as described below.

## Customize API
Your API is derived from the database, but it is not restricted to that.  You can customize your API, e.g., add new endpoints.

### Trivial APIs: Hello World
For example - an instant API for a database is all well and good - but can we build the "hello world" so desperately sought by enterprises all over the world?  We can.

In the screen shot below, examine the following code for ```hello_world()``` in `api_logic_server/api/expose_services.py`

You can run this with curl:
```bash
curl -X GET "http://localhost:5000/hello_world?user=ApiLogicServer"
```

### Services: Add Order
The ```ServicesEndpoint``` class in the screen shot below created a **custom service** for adding an `Order` and an `OrderDetailList`.  Let's see it in action.

Execute this cURL command in your console.  The absurdly high quantity provokes an **intentional constraint failure**, so we can explore logic:

```bash
curl -X POST "http://localhost:5000/ServicesEndPoint/add_order" \
-H  "accept: application/vnd.api+json" -H  "Content-Type: application/json" \
-d '{
      "meta": {
        "method": "add_order",
        "args": {
          "CustomerId": "ALFKI",
          "EmployeeId": 1,
          "Freight": 10,
          "OrderDetailList": [
            {
              "ProductId": 1,
              "Quantity": 1111,
              "Discount": 0
            },
            {
              "ProductId": 2,
              "Quantity": 2,
              "Discount": 0
            }
          ]
        }
      }
    }'

```

and observe the response (the exception is intended).

The implementation starts on line 99:

<figure><img src="https://github.com/valhuber/ApiLogicServer/blob/main/images/tutorial/services-reuse.png?raw=true"></figure>


In the code above:

1. **Service Definition:** to define a service:

   a. Edit ```expose_services.py```, and define a **decorated method**.  The payload (`Order` and a list of `OrderDetails`) is passed in as `*kwargs`.

   b. Your first comment **describes the arguments** in YAML (not shown), along with sample data (you can use this in _try it_).  The system passes this along to the Swagger.

2. **Service Processing**

   a. Use standard Python for your service code, including SQLAlchemy and other Python packages. (Note the use of `json_to_entities`, to transform the json payload into mapped SQLAlchemy model objects.  It is **generic**, so can be used with your own model objects).

   b. The code is very short since the logic is **automatically reused**.  The rules that fired for changing a Customers' CreditLimit (left pane - these are the executable rules) also monitor order processing (right pane - this is the console log).   The indentation illustrates the **chaining of a multi-table transaction:**

<figure><img src="https://github.com/valhuber/ApiLogicServer/blob/main/images/tutorial/order-logic-log.png?raw=true"></figure>

> **Key Takeway: Automatic Reuse:** In traditional procedural coding, re-use is generally achieved with significant design work.  But in this declarative approach, logic is _automatically_ _**re-used**_ for all transactions.  That has a significant impact on conciseness (5 rules vs 200 lines of code), and quality.




## Customize Logic

Use Python to customize logic.  Let's POST a new Order:

1. Paste the following into the right Terminal window (same `add_order` post as above, but with an acceptable quantity):

```bash
curl -X POST "http://localhost:5000/ServicesEndPoint/add_order" \
-H  "accept: application/vnd.api+json" -H  "Content-Type: application/json" \
-d '{
      "meta": {
        "method": "add_order",
        "args": {
          "CustomerId": "ALFKI",
          "EmployeeId": 1,
          "Freight": 10,
          "OrderDetailList": [
            {
              "ProductId": 1,
              "Quantity": 1,
              "Discount": 0
            },
            {
              "ProductId": 2,
              "Quantity": 2,
              "Discount": 0
            }
          ]
        }
      }
    }'

```

2. Observe the log entry containing *Congratulations* (again, viewed without word wrap):

<figure><img src="https://github.com/valhuber/ApiLogicServer/blob/main/images/tutorial/custom-logic-logic-log.png?raw=true"></figure>


The log entry is generated by the following in `api_logic_server/logic/logic_bank.py`:

```python
    def congratulate_sales_rep(row: models.Order, old_row: models.Order, logic_row: LogicRow):
        if logic_row.ins_upd_dlt == "ins":  # logic engine fills parents for insert
            sales_rep = row.Employee
            if sales_rep is None:
                logic_row.log("no salesrep for this order")
            else:
                logic_row.log(f'Hi, {sales_rep.Manager.FirstName} - '
                              f'Congratulate {sales_rep.FirstName} on their new order')
    # ...
    
    Rule.commit_row_event(on_class=models.Order, calling=congratulate_sales_rep)  # declare logic event

```

> **Key Takeaway:** ApiLogicServer spreadsheet-like logic is **powerful (40X),** and **extensible** with standard Python (see links for more information.

## Customize Web App
The Basic Web App is driven by `ui/basic_web_app/app/views.py`, which contains classes like this for each table:

```python
class CustomerModelView(ModelView):
   datamodel = SQLAInterface(Customer)
   list_columns = ["CompanyName", "ContactName", "ContactTitle", "Address", "City"]
   show_columns = ["CompanyName", "ContactName", "ContactTitle", "Address", "City", "Region", "PostalCode", "Country", "Phone", "Fax", "Balance", "CreditLimit", "OrderCount", "Id", "UnpaidOrderCount"]
   edit_columns = ["CompanyName", "ContactName", "ContactTitle", "Address", "City", "Region", "PostalCode", "Country", "Phone", "Fax", "Balance", "CreditLimit", "OrderCount", "Id", "UnpaidOrderCount"]
   add_columns = ["CompanyName", "ContactName", "ContactTitle", "Address", "City", "Region", "PostalCode", "Country", "Phone", "Fax", "Balance", "CreditLimit", "OrderCount", "Id", "UnpaidOrderCount"]
   related_views = [OrderModelView]

appbuilder.add_view(
      CustomerModelView, "Customer List", icon="fa-folder-open-o", category="Menu")
```

You can edit this file to control what columns are displayed, their display order, and what related data (views) are shown.

> Flask AppBuilder has a wide range of capabilities, including charts.  For more information, see [Flask AppBuilder](https://flask-appbuilder.readthedocs.io/en/latest/).


> **Key Takeaway:** ApiLogicServer multi-page, multi-table apps provide instant back-office admin and agile prototyping.

# ApiLogicServer Tutorial - Wrap-up

In **20 minutes - instead of weeks or months** - you have built and executed an **ApiLogicServer**, and explored its 3 key elements:


| Element | Key Takeaway   | Why It Matters  |
| :-------------- |:--------------| :------| 
| JSON:**API**  | Created **instantly**, *customizable* | Unblock UI Development |
| **Logic**| Spreadsheet-like rules automate the cocktail napkin spec | **40X more concise**, *customizable* with Python  |
| Basic **Web App** | Multi-page, multi-table - created **instantly**, customizable  | Back-office admin apps, agile prototyping |

That's a significant result. This combination of an instant API and spreadsheet-like logic - customizable with Python - may mean its time to take a very close look at this technology, to create database backends with agility that is strategic to the business.

## Links

1. Orientation

   * You are in a [JupyterLab Notebook](https://jupyterlab.readthedocs.io/en/stable/).

   * Installation has already been completed for both
[Python](https://github.com/valhuber/ApiLogicServer/wiki/Python-Verify-and-Install) and [ApiLogicServer](https://github.com/valhuber/ApiLogicServer#installation).
ApiLogicServer installs with a [Northwind sample database](Sample-Database), which we'll be using in this Tutorial.

   * If you are considering Jupyter notebooks for your own no-install demos,  you can explore how this demo was built on [GitHub](https://github.com/valhuber/ApiLogicServerTutorial).
   
2. Create ApiLogicServer

   * Here is how to specify a [SQLAlchemy url](https://docs.sqlalchemy.org/en/14/core/engines.html)
   
3. Open API (Swagger)

   * [proxy](https://discourse.jupyter.org/t/how-to-redirect-in-repo2docker-jupyter-server-proxy-mybinder-containers/5209) issue under investigation


   
4. Logic - Explore Types, Examples

   * Here is a complete [list of rule types.](https://github.com/valhuber/LogicBank/wiki/Rule-Summary)

   * To see rules in action, explore the [Examples.](https://github.com/valhuber/LogicBank/wiki/Examples)
   
   * For a general background on logic, see the [Logic Bank Overview](https://github.com/valhuber/LogicBank/wiki)
   
5. Logic - Perspective: 40X

   *  See the [logic vs code contrast here](https://github.com/valhuber/LogicBank/wiki/by-code)
   
6. Explore Basic Web App

   * Before running, you must [Create Admin Data](https://github.com/valhuber/ApiLogicServer/wiki/Working-with-Flask-AppBuilder) for Flask App Builder (except for Northwind, which is pre-created)
   
   
7. Customize Logic

    * You can [leverage the full power of Python](https://github.com/valhuber/LogicBank/wiki/Rule-Extensibility)
for events, and more complex constraints and formulas
